In [1]:
import tensorflow as tf
import numpy as np

/home/dluser1/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
a = tf.get_variable("a", [1])
b = tf.get_variable("b", [1])
Y = tf.convert_to_tensor([a + b, 4*a - b, 5*a + 3*b], name="Y")
print(Y)
y_grad_a = tf.gradients(Y, [a])
y_grad_b = tf.gradients(Y, [b])

Tensor("Y:0", shape=(3, 1), dtype=float32)


In [3]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(Y))
    print(sess.run(y_grad_a))
    print(sess.run(y_grad_b))

[[ 2.7781806]
 [ 2.5253758]
 [10.455965 ]]
[array([10.], dtype=float32)]
[array([3.], dtype=float32)]


In [4]:
# refer to https://stackoverflow.com/questions/51858970/tf-gradients-sums-over-ys-does-it
x_dims = 3
batch_size = 4

x = tf.placeholder(tf.float32, (None, x_dims))

y = 2*(x**2)

grads = tf.gradients(y,x)

sess = tf.Session()

x_val = np.random.randint(0, 10, (batch_size, x_dims))
y_val, grads_val = sess.run([y, grads], {x:x_val})

print('x = \n', x_val)
print('y = \n', y_val)
print('dy/dx = \n', grads_val[0])

x = 
 [[7 9 5]
 [0 5 1]
 [9 6 6]
 [7 8 5]]
y = 
 [[ 98. 162.  50.]
 [  0.  50.   2.]
 [162.  72.  72.]
 [ 98. 128.  50.]]
dy/dx = 
 [[28. 36. 20.]
 [ 0. 20.  4.]
 [36. 24. 24.]
 [28. 32. 20.]]


In [5]:
# refer to https://github.com/tensorflow/tensorflow/issues/675
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/parallel_for/gradients.py#L28
# https://uesugierii.github.io/articles/9.html
from tensorflow.python.ops.parallel_for import jacobian, batch_jacobian

In [6]:
Y_ab = tf.convert_to_tensor(jacobian(Y, [a, b]))
Y_a = tf.squeeze(tf.convert_to_tensor(jacobian(Y, a)))
print(Y_a)
print(Y_ab)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    result = sess.run(Y_ab)
    res = []
    for i in range(len(result)):
        res.append(np.squeeze(result[i].tolist()))
    
    result_a = sess.run(Y_a)
    print(result_a)

Tensor("Squeeze:0", shape=(3,), dtype=float32)
Tensor("packed:0", shape=(2, 3, 1, 1), dtype=float32)
[1. 4. 5.]


In [7]:
res

[array([1., 4., 5.]), array([ 1., -1.,  3.])]